# [Logo]

Quella che segue è una descrizione di massima di una "famiglia" di progetti di **Tipo A** che possono essere concordati con il docente per il primo appello d'esame.

Il progetto richiede l'implementazione di un interprete per il [linguaggio Logo](https://en.wikipedia.org/wiki/Logo_(programming_language)) (così come dettagliato nella sezione seguente). A tal fine:

* deve essere realizzata una **grammatica** derivante dalle specifiche illustrate di seguito (eventualmente prendendo spunto dalle grammatiche [ucb-logo](https://github.com/antlr/grammars-v4/tree/master/ucb-logo), o [logo](https://github.com/antlr/grammars-v4/blob/master/logo));
* deve essere implementato un **parser**, costruito manualmente, o utilizzando ANTLR;
* deve essere implementato un **interprete**, di tipo ricorsivo, o iterativo (basato su *code threading*).

L'interprete può fare uso di una *runtime* per la parte grafica come:

* il pacchetto [turtle](https://docs.python.org/3.3/library/turtle.html?highlight=turtle) delle API standard di Python,
* il pacchetto [tartaruga](https://github.com/mapio/tartaruga) o analogo per Java,
* il codice in [papert](https://github.com/rmmh/papert) o analogo per JavaScipt.

A seconda delle scelte, è possibile ridurre la porzione di specifiche del linguaggio Logo che l'interprete deve implementare.

Da giovedì 30 maggio è possibile iniziare a discutere col docente l'assegnazione del progetto.

## Specifiche del linguaggio Logo

Le specifiche del linguaggio Logo a cui fare riferimento nello svolgimento del progetto sono basate sul [Berkeley Logo User Manual](https://people.eecs.berkeley.edu/~bh/usermanual) [BLUM]; a ciascuna *sezione* di tale manuale (indicata in esso da una sottolineatura composta di segni `=`) corrisponde qui una sezione che chiarisce in che termini essa debba essere rispettata dall'interprete.


### GETTER/SETTER VARIABLE SYNTAX

Le considerazioni di questa sezione del BLUM sono sostanzialmente da ignorare.

Si assuma viceversa che l'interprete abbia un unico spazio dei nomi per *variabili* e *procedure*. Per quanto concerne l'*invocazione delle procedure*, usualmente in Logo  essa avviene indicando il nome della procedura seguito dalle espressioni corrispondenti ai suoi *input* (o argomenti). Dato che le procedure *primitive* (ossia che sono parte del linguaggio, o predefinite) hanno un numero di input noto a priori, non pongono problemi di parsing; questo non è vero per le procedure definite dall'utente (come si vedrà, tramite la procedura di "forma speciale" `TO`). 

Se `F` e `G` sono due procedure il cui numero di *input* non è noto, l'espressione `F G 1 2 3` può essere intesa come `F (G 1 2 3)`, o `F (G 1 2) 3`, oppure `F (G 1) 2 3`.

Per risolvere questo problema in modo semplice, l'interprete può assumere che (diversamente dal Logo descritto nel BLUM), **l'invocazione di una procedura definita dall'utente che abbia un numero di argomenti diverso da 1 sia racchiusa tra parentesi** (un po' come avviene per le procedure `SUM` e `PRODUCT` quando il numero di input è maggiore di 2). 

Con tale restrizione, l'espressione `F G 1 2 3` diventa illegittima (non è possibile che sia `F` che `G` siano unarie), le tre interpretazioni possibili elencate sopra devono essere indicate come `F (G 1 2 3)`, o `(F (G 1 2) 3)`, oppure `(F G 1 2 3)` (a seconda che, rispettivamente, `F` sia unaria, siano entrambe binarie, o `G` sia unaria).


### ENTERING AND LEAVING LOGO

Questa sezione del BLUM è da ignorare. L'interprete deve essere implementato come una funzione che prende in ingresso una stringa corrispondente al codice Logo e non restituisce alcun valore (ma produce l'eventuale disegno descritto nel codice).


### TOKENIZATION

Le considerazioni descritte in questa sezione del BLUM sono da ignorare in favore di una versione molto più semplificata del processo di divisione in *token* descritta di seguito. 

Assumete che gli *identificatori* (nomi di *procedure* e *variabili*) siano composti da una sequenza non vuota di lettere alfabetiche, che i *numeri* (interi e reali) abbiano l'usuale struttura lessicale e che le *stringhe* siano costituite dal carattere `"` seguito da uno o più caratteri alfanumerici (spazio escluso). Sono da considerarsi riservati gli identificatori `END` (usato come terminatore della "forma speciale" per la procedura `TO` di cui si parla più avanti) e la coppia`TRUE` e `FALSE` che corrisponde ai letterali per i tipi *booleani* vero e falso.

Assumete che i *token* siano separati dall'usuale *whitespace* (spazi, tabulazioni e "a capo") e che quindi più invocazioni di procedura possano susseguirsi sulla stessa linea.


### DATA STRUCTURE PRIMITIVES

Questa sezione del BLUM è da ignorare; oltre a numeri e stringhe, l'unica struttura non elementare che l'interprete deve gestire sono le liste letterali (ossia presenti nel sorgente e delimitate da `[` e `]`), richieste come tipo di input da alcune procedure.


### COMMUNICATION 

Di questa sezione del BLUM l'interprete deve implementare solo la procedura `PRINT`.


### ARITHMETIC

Di questa sezione del BLUM l'interprete deve implementare tutte le procedure elencate in `NUMERIC OPERATIONS` tranne `ISEQ` e `RSEQ` (fate attenzione alla precedenza del `-`/`MINUS` unario), tutti i predicati in `PREDICATES` e le procedure `RANDOM NUMBERS`.

L'interprete deve inoltre implementare le versioni infisse delle procedure che le prevedono, prestando particolare attenzione alla loro precedenza e all'associatività.


### LOGICAL OPERATIONS

Di questa sezione del BLUM l'interprete deve implementare tutte le procedure che rappresentano i connettivi e le loro versioni infisse (prestando particolare attenzione alla precedenza, sopratutto in rapporto alle operazioni aritmetiche infisse). Va inoltre rispettata la [short-circuit evaluation](https://en.wikipedia.org/wiki/Short-circuit_evaluation).


### GRAPHICS

Di questa sezione del BLUM l'interprete deve implementare tutte le procedure di `TURTLE MOTION` e di `TURTLE AND WINDOW CONTROL` fino a `CLEARSCREEN`/`CS` (inclusa). Tra le procedure di `PEN AND BACKGROUND CONTROL` deve implementare solo `PENDOWN`/`PD`, `PENUP`/`PU`, `SETPENCOLOR`/`SETPC` (solo nella forma in cui l'input è una lista di valori RGB) e `SETPENSIZE`.


### WORKSPACE MANAGEMENT

Di questa sezione del BLUM l'interprete deve implementare, per la parte di `PROCEDURE DEFINITION`, solo la procedura `TO` (e `END`) con i soli *required input* (ma senza *optional*/*rest* input o numero di input di *default*).  Per la parte di `VARIABLE DEFINITION` solo `MAKE` e `THING`/`:`.


### CONTROL STRUCTURES

Di questa sezione del BLUM l'interprete deve implementare le sole strutture di controllo `REPEAT`, `IF` (nella forma con un solo input), `IFELSE`, `STOP` (si osservi che questa struttura di controllo equivale all'usuale `return` senza espressione, non a un `break`), `OUTPUT`/`OP` e `WHILE`.


### MACROS, ERROR PROCESSING, SPECIAL VARIABLES, INTERNATIONALIZATION

Queste sezioni del BLUM sono da ignorare; riguardo alla gestione degli errori, l'interprete deve quanto meno catturare gli errori di esecuzione del linguaggio in cui è implementato e segnalare gli errori del sorgente Logo o della sua esecuzione in modo il più possibile chiaro (non necessariamente indicando il luogo del sorgente a cui è dovuto l'errore).
